In [164]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

words = open('names.txt','r').read().splitlines()

In [165]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [177]:
block_size = 3

def build_dataset(words):    
    X,Y = [],[]
    for w in words:
        #print(w)
        context = [0]*block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context)+ '-->' + ch)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,Ytr = build_dataset(words[:n1])
Xdev,Ydev = build_dataset(words[n1:n2])
Xte,Yte = build_dataset(words[n2:])

In [182]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn(27,10)
W1 = torch.randn((30,100))
B1 = torch.randn(100)
W2 = torch.randn((100,27))
B2 = torch.randn(27)
parameters = [C,W1,B1,W2,B2]

In [183]:
for p in parameters:
    p.requires_grad = True

In [188]:
for _ in range(50000):
    ix = torch.randint(0,Xtr.shape[0],(32,))#idx for mini batch of 32

    emb = C[Xtr[ix]] # X[ix] selects randon 32 elements
    h = torch.tanh(emb.view(-1,block_size*10) @ W1 + B1)
    logits = h@W2 + B2
    loss = F.cross_entropy(logits,Ytr[ix])

    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.01*p.grad

print(loss.item())

2.420025587081909


In [190]:
emb = C[Xdev] # X[ix] selects randon 32 elements
h = torch.tanh(emb.view(-1,block_size*10) @ W1 + B1)
logits = h@W2 + B2
loss = F.cross_entropy(logits,Ydev)
loss

tensor(2.2016, grad_fn=<NllLossBackward0>)

In [196]:
g= torch.Generator().manual_seed(2147483647+10)

for _ in range(10):
    out = []
    context = [0]*block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1,-1)@W1 + B1)
        logits = h@W2+B2
        probs = F.softmax(logits,dim=1)
        ix = torch.multinomial(probs,num_samples=1,generator=g).item()
        context = context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break
    
    print(''.join(itos[i] for i in out))

carlah.
ami.
hari.
kimlex.
thay.
salaysle.
rahnel.
den.
rha.
kaqui.
